In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome("../../Data Science/chromedriver")

<ipython-input-20-97885315f112>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../../Data Science/chromedriver")


In [119]:
base_url = "http://apartments.com/seattle-wa"
driver.get(base_url)

In [26]:
#search_bar = driver.find_element(By.ID,"quickSearchLookup")

In [125]:
page_range = driver.find_element(By.CLASS_NAME,"pageRange").text
n_pages = int(page_range.split()[-1])

28

In [90]:
all_links = []
for x in range(1,n_pages+1):
    driver.get(f"{base_url}/{x}/")
    link_elements = driver.find_elements(By.CLASS_NAME,'property-link')
    links = [x.get_attribute("href") for x in link_elements]
    all_links.extend(links)

In [91]:
link_1 = driver.get(all_links[0])

In [92]:
built_date = driver.find_element(By.XPATH,"//div[contains( text( ), 'Built in')]").text
built_date

'Built in 2022'

In [107]:
unit_counts = driver.find_elements(By.XPATH,"//div[contains( text( ), 'unit')]")
unit_count = [x.text for x in unit_counts if "available" not in x.text.lower() and 'unit' in x.text.lower()][0]
unit_count

'118 units/5 stories'

In [118]:
address_elem = driver.find_elements(By.CLASS_NAME,"propertyAddressContainer")
address,neighborhood = address_elem[0].text.split("\n")
print(address)
print(neighborhood)

4106 N Stone Way, Seattle, WA 98103
Wallingford
